<div align="center">
    <h2>Reinforcement Learning Summer 2024</h2>
    <h2>Prof. Dr. Frank Kirchner</h2>
    <h4>Exercise Sheet – III</h4>
    <h5>Due: 22.05.24</h5>
    <hr>
</div>


**Problem 3.1: Eligibility Traces (10 P.)**

Using the information from a reward solely to update one state does not seem efficient when we could have updated the states in the path as well. Eligibility Traces (ET) do exactly that. This task is for exploring ET and visualizing the results from using different versions of ET. For that you will use the provided custom grid world.  



<p style="text-align:center;">
<img src="grid_world.png" alt="Your Image" width="400">
</p>

**Hand in the final code of the whole exercise and also the plotted curves for all subtasks.**

1. **(1 P.)** Make yourself familiar with the environment. E.g. The observation- and action-spaces, reward, returns, etc.
2. **(4 P.)** Implement both SARSA($\lambda$) and WatkinsQ($\lambda$) with the option to choose between the methods. Use an e-greedy implementation with $\epsilon = e^{-i/\text{max}(i)}$ where $i$ is the number of the current episode. Choose a good $\alpha$ and $\lambda$.
3. **(1 P.)** Add also the option to switch between replacing and accumulating traces. Run a test with 100 episodes for each version and plot the cumulative rewards.
4. **(2 P.)** Implement the option to print out a path (use the parameter *path* in the render functions). Path should be a list consisting of 3-tuples with the coordinates and the increase in the q-value of the taken actions. Use small circles to mark the path with their size indicating the increase.
5. **(2 P.)** Test the previous implementation with a TD($\lambda$) control method of your choice. Visualize the traces every ten percent of the total episodes. Run this until the agent does not run into walls anymore. Thus, find a good number of training episodes. In the end plot the cumulative reward and mark the cumulative reward with the iteration when each of the depicted trajectories started.


### Defining environment

In [5]:
import gymnasium as gym
from gymnasium import spaces
import pygame
import numpy as np
from gymnasium.envs.registration import register
import warnings

# Filter warnings specifically from gymnasium's core.py module
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium.core")


class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array", "traces"], "render_fps": 0.5}

    def __init__(self, render_mode=None, seed=42):
        self._render = False
        self.path = []
        super().reset(seed=seed)
        self.size = 7  # The size of the square grid
        self.window_size = 512  # The size of the PyGame window

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, self.size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, self.size - 1, shape=(2,), dtype=int),
                "walls": spaces.MultiBinary([self.size, self.size])
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(4)

        # Use fixed target location
        self._target_location = self.np_random.integers([self.size // 2 + 1, 0], [self.size - 1, self.size - 1], 2, dtype=int)
       
        # Create the wall, leave out a gab in the center
        self._walls_location = np.zeros((self.size, self.size), dtype=np.int32)
        self._walls_location[self.size // 2] = 1
        self._walls_location[self.size // 2, self.size // 2] = 0

        """
        The following dictionary maps abstract actions from `self.action_space` to
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location, "walls": self._walls_location}
        
    def _get_info(self):
        return {
            "distance": np.linalg.norm(
                self._agent_location - self._target_location, ord=1
            )
        }
        
    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)
    
        # Choose the agent's location uniformly at random
        self._agent_location = self.np_random.integers([0, 0], [self.size // 2 - 1, self.size - 1], 2, dtype=int)

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def step(self, action):
        reward = 0
        # Map the action (element of {0,1,2,3}) to the direction we walk in
        direction = self._action_to_direction[action]
        # We use `np.clip` to make sure we don't leave the grid

        self._agent_location = np.clip(
            self._agent_location + direction, 0, self.size - 1
        )

        # Test if agent hit a wall -> Negative reward
        if self._walls_location[tuple(self._agent_location)] == 1:
            self._agent_location -= direction
            reward = -0.5

        # An episode is done iff the agent has reached the target
        terminated = np.array_equal(self._agent_location, self._target_location)
        reward = 1 if terminated else reward  
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human" or self._render:
            self._render_frame(reward=reward)

        return observation, reward, terminated, False, info
    
    # support funcions to get around the strict environment specifications
    def set_render(self, render):
        self._render = render 

    def render_path(self, path):
        self._render_frame(path)

    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()
        elif self.render_mode == "traces":
            self._render_frame()

    def _render_frame(self, path=[], reward=0):
        if self.window is None and self.render_mode in ["human", "traces"]:
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (self.window_size, self.window_size)
            )
        if self.clock is None and self.render_mode in ["human", "traces"]:
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.size
        )  # The size of a single grid square in pixels

        # First we draw the target
        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * self._target_location,
                (pix_square_size, pix_square_size),
            ),
        )
        # Now we draw the agent
        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (self._agent_location + 0.5) * pix_square_size,
            pix_square_size / 3,
        )

        # Draw walls 
        # added visualization of reaching the goal (green) and hitting a wall (red)
        if reward == 1:
            rgb = (0, 255, 0)
        elif reward == -0.5:
            rgb = (255, 0, 0)
        else:
            rgb =(0, 0, 0)

        for i in range(self.size):
            for j in range(self.size):
                if self._walls_location[i,j] == 1:
                    pygame.draw.rect(
                        canvas,
                        rgb,
                        [pix_square_size * i, pix_square_size * j, pix_square_size, pix_square_size]
                        )


        # draw path if it is not empty
        if path:
            # get max delta
            dq_max = max(path[2::3])
            dq_min = min(path[2::3])
            for i in range(len(path) // 3):
                # get three tuples 
                x, y, dq = path[i*3:i*3+3]
                # only plot legit dqs
                if dq > 0:
                    pygame.draw.circle(
                    canvas,
                    (0, 200, 0),
                    ((x + 0.5) * pix_square_size, (y + 0.5) * pix_square_size),
                    dq / dq_max * pix_square_size / 3,
                    )
                elif dq < 0:
                    pygame.draw.circle(
                    canvas,
                    (200, 0, 0),
                    ((x + 0.5) * pix_square_size, (y + 0.5) * pix_square_size),
                    dq / dq_min * pix_square_size / 3,
                    )


        # Finally, add some gridlines
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        if self.render_mode in ["human", "traces"]:
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        
    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


### 3.1a: Eligiblity traces intialisation

In [2]:
import gymnasium as gym
env = GridWorldEnv(render_mode="traces")
observation, info = env.reset(seed=42)
for _ in range(1000):
   action = env.action_space.sample() # User-defined policy function
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()

### 3.2b: Run eligiblity traces

In [ ]:
import gymnasium as gym

env = GridWorldEnv(render_mode="traces")
env.reset()

for i in range(100):
    a = env.action_space.sample()
    obs, r, terminated, truncated, info = env.step(a)
    
    if terminated:
        env.reset()



**Problem 3.2: Model-based Reinforcement Learning (10 P.)**

In the last sheet we used Q-Learning and SARSA to find the optimal way to navigate the frozen lake environment. However, the 8x8 version of the environment took much longer than the 4x4 one. Reasons for this were:

- Constant epsilon value
- Reward only when transitioning to the goal state
- Update of values only based on the previous values
- "Back and forth" during exploration

For this task we want to improve the results a lot by using the model-based approaches from the lecture.

**Hand in the final code of the whole exercise and also the plotted curves for all subtasks.**

1. **(3 P.)** Start by implementing the plain DynaQ-Algorithm. The function should return the final Q-values as well as a list with the cumulative reward for each iteration. Test your implementation with these parameter values:
    - $epsilon = 0.5$ (you can also try out other values for epsilon)
    - $gamma = 0.9$, $alpha= 0.1$, $N = 10$ (Planning index)
    - Initial Q-values $init\_value = 0.0$ and 
    - Iterations = 200000
    When plotting the results with the provided function you most probably will observe that all the Q-values are still zero. Change the constant value for epsilon to the dynamic $\epsilon = e^{-i/\text{max}(i)}$ where i is the current number of iterations. You might need to run this a few times until you see a result but it should find the goal. Since reaching the goal is not guaranteed and the number of iterations is enormous you are going to improve the algorithm to enhance the results.
2. **(3 P.)** The rewards here are sparse and the environment is deterministic, therefore add Prioritized Sweeping to the implementation  (use the theta parameter in the function call). You do not need to implement a complex priority queue. Two lists are sufficient. Also, the defaultdict might help you with step 7 d). Test this with the parameters $gamma = 0.9$, $alpha = 0.5$, $N = 10$ (maximum planning iterations) and $theta = 0.01$. What do you observe?
3. **(2 P.)** Try now to add a negative reward every time the agent falls into an ice hole. Implement a [custom wrapper](https://gymnasium.farama.org/tutorials/gymnasium_basics/implementing_custom_wrappers/) from the gymnasium framework that adds a settable negative reward r_hole when the agent falls into an ice hole. Test this with r_hole values of [0, -0.01, -0.05, -0.1, -0.5, -1, -2] and the same parameters as before, recording the cumulative rewards for the episodes. Plot the curves for the rewards.
4. **(2 P.)** At last, incorporate the idea of balanced wandering into the $\epsilon$-greedy policy. Right now, exploration means choosing a random action. Your version should instead choose the action that was chosen the least so far when the state is unknown (in the sense of the the $E^3$-Algorithm). Also test this with different values and parameters of your choice. Hand in the plot of the parameter combination with the best result using the provided plotting function.

*Optional:* Try out different frozen lake maps by using the random map generator:

```python
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
gym.make('FrozenLake-v1', desc=generate_random_map(size=8, seed=5))


### 3.2a: Initialising Model based RL

In [8]:
# You do not need to use the already imported modules, but they might help you
import numpy as np
import random
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt

# Hint for using defaultdict
#
# You can play around with the following few lines 
#
# from collections import defaultdict
# test = defaultdict(dict)
# test["a"]["a"] = 5
# test["a"]["a"] = 6
# test["a"]["b"] = 6
# test["b"]["b"] = 6
# print(test)

class MBLearner:

    # suggested init
    def __init__(self, env, size = (8,8)):
        self.env = env
        self.size = size
        self.rng = np.random.default_rng()

    def e_greedy(self, state, epsilon, wander):
        # to be implemented
        action = None
        return action
                       
    # Keep stubs the same!
    def learning(self, iterations, N, balanced_wandering=0, theta = 0.1, alpha = 0.5, gamma = 0.9):
        # to be implemented
        
        # presets for plotting
        # Keep type and dimensions the same!
        self.reward = [0]
        self.q_values = np.zeros((self.size[0] * self.size[1], self.env.action_space.n))

        return self.q_values.copy(), self.reward


    ###############################
    #       Only for plotting
    ###############################
    def plot_q_values_map(self):
        """Plot the last frame of the simulation and the policy learned."""
        qtable_val_max, qtable_directions = self._qtable_directions_map(self.size)

        # Plot the last frame
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
        ax[0].imshow(self.env.render())
        ax[0].axis("off")
        ax[0].set_title("Last frame")

        # Plot the policy
        sns.heatmap(
            qtable_val_max,
            annot=qtable_directions,
            fmt="",
            ax=ax[1],
            cmap=sns.color_palette("Blues", as_cmap=True),
            linewidths=0.7,
            linecolor="black",
            xticklabels=[],
            yticklabels=[],
            annot_kws={"fontsize": "xx-large"},
        ).set(title="Learned Q-values\nArrows represent best action")
        for _, spine in ax[1].spines.items():
            spine.set_visible(True)
            spine.set_linewidth(0.7)
            spine.set_color("black")
        img_title = f"frozenlake_q_values_{self.size[0]}x{self.size[1]}.png"
        fig.savefig(img_title, bbox_inches="tight")

        # Plot the cumulative reward
        ax[2].plot(self.reward)
        ax[2].set_xlabel("Iterations")
        ax[2].set_title("Cumulative reward")
        ax[2].axhline(0, 0, len(self.reward), linestyle="dashed", color="black")

        plt.show()

    def _qtable_directions_map(self, map_size):
        """Get the best learned action & map it to arrows."""
        qtable_val_max = self.q_values.max(axis=1).reshape(map_size)
        qtable_best_action = np.argmax(self.q_values, axis=1).reshape(map_size)
        directions = {0: "←", 1: "↓", 2: "→", 3: "↑"}

        qtable_directions = np.empty(qtable_best_action.flatten().shape, dtype=str)
        eps = np.finfo(float).eps  # Minimum float number on the machine
        for idx, val in enumerate(qtable_best_action.flatten()):
            if qtable_val_max.flatten()[idx] > eps or qtable_val_max.flatten()[idx] < -eps:
                # Assign an arrow only if a minimal Q-value has been learned as best action
                # otherwise since 0 is a direction, it also gets mapped on the tiles where
                # it didn't actually learn anything
                qtable_directions[idx] = directions[val]
        qtable_directions = qtable_directions.reshape(map_size)
        return qtable_val_max, qtable_directions

### 3.2b: Run model based RL

In [ ]:
import gymnasium as gym
from matplotlib import pyplot as plt


# initialize base environment
base_env = gym.make('FrozenLake-v1', render_mode="rgb_array", desc=None, map_name="8x8", is_slippery=False)
base_env.close()


---
Please upload your submission via StudIP by 20:00 on May 22, 2024. If you encounter any issues with the upload process, please contact me in advance at laux@uni-bremen.de.

Your submission must include: 
- All the files you created or modified for your submission
- A small .txt file with the names and e-mail addresses of the contributing team members

